## Merging/Cleaning Bike Data

The aim of this script is to create a reusable tool for merging and cleaning New York City Bike data (https://www.citibikenyc.com/system-data) into a single csv file for consumption via Tableau. 
First I want to get a list of the files currently sitting in the data folder, we'll use the **os** module for this task

### Kevin's Additions
+ Prevent double writing first script
+ Age feature
+ Distance feature
+ Zipcode feature... kinda

In [1]:
import pandas as pd

import geopy.distance
from geopy.geocoders import Nominatim

from os import listdir
from os.path import isfile, join
datafiles = [f for f in listdir("data/") if isfile(join("data/", f))]
print(datafiles)

['201704-citibike-tripdata.csv', '201708-citibike-tripdata.csv', '201712-citibike-tripdata.csv', '201804-citibike-tripdata.csv']


## Now lets read in the first file in order to get an idea of what we are working with

In [2]:
df = pd.read_csv("data/" + datafiles[0], parse_dates=True, index_col='starttime')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1315404 entries, 2017-04-01 00:00:58 to 2017-04-30 23:59:53
Data columns (total 14 columns):
tripduration               1315404 non-null int64
stoptime                   1315404 non-null object
start station id           1315404 non-null int64
start station name         1315404 non-null object
start station latitude     1315404 non-null float64
start station longitude    1315404 non-null float64
end station id             1315404 non-null int64
end station name           1315404 non-null object
end station latitude       1315404 non-null float64
end station longitude      1315404 non-null float64
bikeid                     1315404 non-null int64
usertype                   1315404 non-null object
birth year                 1146732 non-null float64
gender                     1315404 non-null int64
dtypes: float64(5), int64(5), object(4)
memory usage: 150.5+ MB


### Great now we just need to pop out that first index so we don't duplicate it on our df

In [3]:
del datafiles[0]
print(datafiles)

['201708-citibike-tripdata.csv', '201712-citibike-tripdata.csv', '201804-citibike-tripdata.csv']


### Ok so it looks like we have a pretty set array of data, now we'll loop through the list of csv files and append them to the file we have above

In [4]:
for csv in datafiles:
    new_df = pd.read_csv("data/" + csv, parse_dates=True, index_col='starttime')
    #print(new_df.head(5))
    df = pd.concat([df, new_df])
    print(len(df))

3131902
4021869
5329390


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5329390 entries, 2017-04-01 00:00:58 to 2018-04-30 23:59:35
Data columns (total 15 columns):
bikeid                     int64
birth year                 float64
end station id             int64
end station latitude       float64
end station longitude      float64
end station name           object
gender                     int64
name_localizedValue0       object
start station id           int64
start station latitude     float64
start station longitude    float64
start station name         object
stoptime                   object
tripduration               int64
usertype                   object
dtypes: float64(5), int64(5), object(5)
memory usage: 650.6+ MB


# Cleaning up the data and sorting by date

In [6]:
df['Gender'] = df['gender'].replace([0,1,2], ['unknown', 'male', 'female'])
# df[['bikeid', 'birth year', 'end station id', 'end station latitude', 'end station longitude', 'end station name', 'Gender', 'name_localizedValue0', 'start station id', 'start station latitude', 'start station longitude', 'start station name', 'stoptime', 'tripduration', 'usertype']]

### Now let's engineer Age, by taking the ride stop time year and subtracting birth year if it exists, as well as Distance

In [7]:
df['Age'] = pd.to_numeric(df['stoptime'][0][:4]) - df['birth year']

### Let's get zip codes and this might be the fastest way to do it?

### Add in the stations in our end station section to grab all of those zip codes!

### And save it!!!!  This took a decent amount of time to do

#### Turning it into something a bit more useable

#### And then I ran into an issue with missing one zipcode and class started and I didn't feel like finishing it

#### OK So this is how to add everything once it's all working nicely

In [79]:
# This takes a little time
distances = []
start_station_zips = []
end_station_zips = []
start_stations = []
end_stations = []
for index, row in df.iterrows():
    start_coords = [row['start station latitude'], row['start station longitude']]
    end_coords = [row['end station latitude'], row['end station longitude']]
    distances.append(geopy.distance.distance(start_coords, end_coords).miles)
    start_station_zips.append(station_zips[station_names.index(row['start station name'])])
    end_station_zips.append(station_zips[station_names.index(row['end station name'])]) 

ValueError: '9 Ave & W 18 St' is not in list

#### ^^^^Working nicely ^_^

In [41]:
df['Distance'] = distances
df['Start Station Zipcode'] = start_station_zips
df['End Station Zipcode'] = end_station_zips

0.000000    108957
1.480843      2945
3.039488      2907
0.501124      2774
0.296588      2599
2.559179      2508
0.249226      2476
0.708702      2462
0.485263      2351
0.425562      2351
0.703516      2323
0.472443      2262
0.357127      2232
0.325062      2206
0.870129      2200
0.682090      2160
2.331461      2116
1.383191      2112
0.650809      2096
0.590944      2081
0.405118      2054
0.449154      1994
0.583632      1977
0.839115      1947
0.689726      1925
0.230075      1922
0.718156      1895
0.505528      1862
0.889393      1853
0.430139      1842
             ...  
5.044555         1
2.493808         1
4.995481         1
3.701005         1
4.438464         1
3.173937         1
2.117186         1
4.308691         1
5.839255         1
4.705512         1
3.032099         1
2.078960         1
3.160625         1
2.992339         1
4.546876         1
1.821777         1
4.390918         1
3.391037         1
1.881611         1
2.005592         1
3.100143         1
2.503653    

### Continue Running Code From This Point
+ Point

In [ ]:
# This takes a little time
distances = []
for index, row in df.iterrows():
    start_coords = [row['start station latitude'], row['start station longitude']]
    end_coords = [row['end station latitude'], row['end station longitude']]
    distances.append(geopy.distance.distance(start_coords, end_coords).miles)
df['Distance'] = distances

In [42]:
df = df[['Age', 
         'bikeid', 
         'birth year', 
         'Distance', 
         'end station id', 
         'end station latitude', 
         'end station longitude', 
         'end station name', 
         'End Station Zipcode', 
         'Gender', 
         'name_localizedValue0', 
         'start station id', 
         'start station latitude', 
         'start station longitude', 
         'start station name', 
         'Start Station Zipcode',
         'stoptime', 
         'tripduration', 
         'usertype']]

#### I think maybe some people lied about their birth year...

In [43]:
df['Age'].unique()

array([  51.,   22.,   30.,   56.,   50.,   43.,   46.,   40.,   24.,
         41.,   44.,   28.,   53.,   32.,   27.,   37.,   55.,   31.,
         23.,   67.,   48.,   38.,   20.,   21.,   26.,   35.,   29.,
         63.,   25.,   58.,   19.,   64.,   18.,   nan,   45.,   33.,
         60.,   47.,   36.,   34.,   42.,   39.,   49.,   52.,   57.,
         59.,   69.,   68.,   62.,   61.,   54.,   66.,   65.,   70.,
         17.,   71.,   75.,  100.,   81.,   73.,   74.,  132.,   72.,
         76.,   77.,  117.,   85.,   91.,   78.,   79.,   80.,  124.,
         16.,   83.,   87.,  116.,  131.,   86.,   84.,  107.,  105.,
        110.,   96.,   94.,  129.,  118.,   82.,  122.,   99.,  121.,
         90.,  104.,  130.,   89.,   88.,  102.,   97.,  112.,  101.,
        127.,  123.,  128.,   15.,   98.,  120.])

In [44]:
df['Age'].value_counts()

28.0     203944
29.0     203522
27.0     202265
30.0     194997
32.0     194020
31.0     192732
26.0     188095
33.0     174478
25.0     170519
34.0     163605
35.0     153495
24.0     143085
36.0     142145
37.0     128350
38.0     120355
39.0     111850
40.0     104328
41.0     100815
47.0      98108
42.0      95641
43.0      93878
23.0      93301
46.0      90187
48.0      87717
44.0      87368
45.0      86109
49.0      80236
50.0      75603
52.0      72079
53.0      71046
          ...  
118.0       106
116.0        94
107.0        94
105.0        90
86.0         86
84.0         75
100.0        73
96.0         67
99.0         65
123.0        55
129.0        54
124.0        49
131.0        48
91.0         43
88.0         39
87.0         32
110.0        28
122.0        25
89.0         20
98.0         14
121.0         6
127.0         4
102.0         4
101.0         4
104.0         4
112.0         3
120.0         2
97.0          2
90.0          1
128.0         1
Name: Age, Length: 104, 

In [45]:
df.head()

,Age,bikeid,birth year,Distance,end station id,end station latitude,end station longitude,end station name,Gender,name_localizedValue0,start station id,start station latitude,start station longitude,start station name,stoptime,tripduration,usertype
starttime,,,,,,,,,,,,,,,,,
2017-04-01 00:00:58,51.0,25454,1966.0,0.391795,430,40.701485,-73.986569,York St & Jay St,male,NaN,217,40.702772,-73.993836,Old Fulton St,2017-04-01 00:04:14,195,Subscriber
2017-04-01 00:01:54,22.0,25930,1995.0,1.507976,3158,40.771639,-73.982614,W 63 St & Broadway,male,NaN,490,40.751551,-73.993934,8 Ave & W 33 St,2017-04-01 00:11:40,585,Subscriber
2017-04-01 00:01:56,30.0,27110,1987.0,1.310112,412,40.715816,-73.994224,Forsyth St & Canal St,male,NaN,504,40.732219,-73.981656,1 Ave & E 16 St,2017-04-01 00:13:01,665,Subscriber
2017-04-01 00:02:24,56.0,28392,1961.0,0.432312,411,40.722281,-73.976687,E 6 St & Avenue D,male,NaN,445,40.727408,-73.981420,E 10 St & Avenue A,2017-04-01 00:06:47,262,Subscriber
2017-04-01 00:02:36,50.0,19316,1967.0,0.324843,3300,40.665147,-73.976376,Prospect Park West & 8 St,female,NaN,3423,40.661063,-73.979453,West Drive & Prospect Park West,2017-04-01 00:04:50,134,Subscriber


### And save it to a happy huge file if you just use a years worth of NYC data 
##### (also a good cutoff point because the csvs before April 2017 use different column names)
##### The file is 3.3 gigs from 04-2017 through 04-2018.

In [46]:
df.to_csv('data/2017-04-2018-04-citibike-selected-tripdata.csv', encoding='utf-8')